In [57]:
import random
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

from pyod.models.abod import ABOD

from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore')

In [58]:
# Random Seed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(69) # Seed 고정

## Data Load

In [59]:
train = pd.read_csv('./data/train_data.csv')
test = pd.read_csv('./data/test_data.csv')

## Pre-Processing

#### HP

In [60]:
train.loc[train['type'] == 0, 'HP'] = 30
train.loc[train['type'] == 1, 'HP'] = 20
train.loc[train['type'] == 2, 'HP'] = 10
train.loc[train['type'] == 3, 'HP'] = 50
train.loc[train['type'] == 4, 'HP'] = 30
train.loc[train['type'] == 5, 'HP'] = 30
train.loc[train['type'] == 6, 'HP'] = 30
train.loc[train['type'] == 7, 'HP'] = 30

test.loc[test['type'] == 0, 'HP'] = 30
test.loc[test['type'] == 1, 'HP'] = 20
test.loc[test['type'] == 2, 'HP'] = 10
test.loc[test['type'] == 3, 'HP'] = 50
test.loc[test['type'] == 4, 'HP'] = 30
test.loc[test['type'] == 5, 'HP'] = 30
test.loc[test['type'] == 6, 'HP'] = 30
test.loc[test['type'] == 7, 'HP'] = 30

#### kw

In [61]:
train['kw'] = train['HP'] * 0.746
test['kw'] = test['HP'] * 0.746

#### V = P / A

- kw/motor_current

In [62]:
train['volt'] = train['kw'] / train['motor_current']
test['volt'] = test['kw'] / test['motor_current']

#### 토르크 계산식
- HP / rpm

In [63]:
train['torque'] = train['HP'] / train['motor_rpm']
test['torque'] = test['HP'] / test['motor_rpm']

#### 회전수당 진돌

In [64]:
train['회전수_진동'] = train['motor_rpm'] / train['motor_vibe']
test['회전수_진동'] = test['motor_rpm'] / test['motor_vibe']

#### 회전수당 전류

In [65]:
train['회전수_전류'] = train['motor_rpm'] / train['motor_current']
test['회전수_전류'] = test['motor_rpm'] / test['motor_current']

## Train

In [66]:
angle = ABOD()
angle.fit(train)

ABOD(contamination=0.1, method='fast', n_neighbors=5)

## Predict

In [67]:
pred_test = angle.predict(test)

In [68]:
# 정상 (0), 이상 (1)
list(pred_test).count(0), list(pred_test).count(1)

(6507, 882)

## Submission

In [69]:
submit = pd.read_csv('./data/answer_sample.csv')
submit['label'] = pred_test
submit.to_csv('./submit/ABOD_파생변수2.csv', index=False)